In [1]:
# pip install torch --index-url https://download.pytorch.org/whl/cu124

In [2]:
# pip install transformers datasets seqeval

In [3]:
# pip install --upgrade jupyter

In [4]:
# pip install --upgrade ipywidgets

In [5]:
# pip install 'accelerate>=0.26.0'

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import numpy as np
from peft import LoraConfig, get_peft_model
from seqeval.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)

In [7]:
def read_conll_file(file_path):
    with open(file_path, "r") as f:
        content = f.read().strip()
        sentences = content.split("\n\n")
        data = []
        for sentence in sentences:
            tokens = sentence.split("\n")
            token_data = []
            for token in tokens:
                token_data.append(token.split())
            data.append(token_data)
    return data

In [8]:
train_data = read_conll_file("eng.train")
validation_data = read_conll_file("eng.testa")
test_data = read_conll_file("eng.testb")

In [9]:
test_data

[[['-DOCSTART-', '-X-', '-X-', 'O']],
 [['SOCCER', 'NN', 'B-NP', 'O'],
  ['-', ':', 'O', 'O'],
  ['JAPAN', 'NNP', 'B-NP', 'B-LOC'],
  ['GET', 'VB', 'B-VP', 'O'],
  ['LUCKY', 'NNP', 'B-NP', 'O'],
  ['WIN', 'NNP', 'I-NP', 'O'],
  [',', ',', 'O', 'O'],
  ['CHINA', 'NNP', 'B-NP', 'B-PER'],
  ['IN', 'IN', 'B-PP', 'O'],
  ['SURPRISE', 'DT', 'B-NP', 'O'],
  ['DEFEAT', 'NN', 'I-NP', 'O'],
  ['.', '.', 'O', 'O']],
 [['Nadim', 'NNP', 'B-NP', 'B-PER'], ['Ladki', 'NNP', 'I-NP', 'I-PER']],
 [['AL-AIN', 'NNP', 'B-NP', 'B-LOC'],
  [',', ',', 'O', 'O'],
  ['United', 'NNP', 'B-NP', 'B-LOC'],
  ['Arab', 'NNP', 'I-NP', 'I-LOC'],
  ['Emirates', 'NNPS', 'I-NP', 'I-LOC'],
  ['1996-12-06', 'CD', 'I-NP', 'O']],
 [['Japan', 'NNP', 'B-NP', 'B-LOC'],
  ['began', 'VBD', 'B-VP', 'O'],
  ['the', 'DT', 'B-NP', 'O'],
  ['defence', 'NN', 'I-NP', 'O'],
  ['of', 'IN', 'B-PP', 'O'],
  ['their', 'PRP$', 'B-NP', 'O'],
  ['Asian', 'JJ', 'I-NP', 'B-MISC'],
  ['Cup', 'NNP', 'I-NP', 'I-MISC'],
  ['title', 'NN', 'I-NP', 'O'],
 

In [10]:
def convert_to_dataset(data, label_map):
    formatted_data = {"tokens": [], "ner_tags": []}
    for sentence in data:
        tokens = [token_data[0] for token_data in sentence]
        ner_tags = [label_map[token_data[3]] for token_data in sentence]
        formatted_data["tokens"].append(tokens)
        formatted_data["ner_tags"].append(ner_tags)
    return Dataset.from_dict(formatted_data)

In [11]:
label_list = sorted(
    list(set([token_data[3] for sentence in train_data for token_data in sentence]))
)
label_map = {label: i for i, label in enumerate(label_list)}

In [12]:
print(label_list)
print(label_map)

['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']
{'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 4, 'I-MISC': 5, 'I-ORG': 6, 'I-PER': 7, 'O': 8}


In [13]:
train_dataset = convert_to_dataset(train_data, label_map)
validation_dataset = convert_to_dataset(validation_data, label_map)
test_dataset = convert_to_dataset(test_data, label_map)

In [14]:
datasets = DatasetDict(
    {
        "train": train_dataset,
        "validation": validation_dataset,
        "test": test_dataset,
    }
)

In [15]:
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [16]:
# Set random seeds for reproducibility
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # Ensures deterministic behavior
    torch.backends.cudnn.benchmark = False     # Disables auto-tuning for convolutional layers

In [17]:
from transformers.models.llama.modeling_llama import LlamaModel

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Create custom LlamaModel with bidirectional attention
class LlamaBidirectionalModel(LlamaModel):
    def _update_causal_mask(self, attention_mask):
        # Create bidirectional attention mask (all ones)
        bsz, seq_len = attention_mask.shape
        mask = torch.ones((bsz, 1, seq_len, seq_len), dtype=torch.bool, device=attention_mask.device)
        return mask

# Initialize model with bidirectional attention
model = AutoModelForTokenClassification.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    num_labels=len(label_list)
)

# Replace the base model with bidirectional version
model.base_model = LlamaBidirectionalModel(model.config)
model.config.is_decoder = False
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Verify model device placement
print(f"Model is on device: {next(model.parameters()).device}")

# Optional: Enable CUDA optimizations
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

Model is on device: cuda:0


In [19]:
set_seed(42)

In [20]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=2)

    # Remove special tokens
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "accuracy": accuracy_score(true_predictions, true_labels),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "classification_report": classification_report(true_labels, true_predictions),
    }

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens are ignored during training
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14987 [00:00<?, ? examples/s]

Map:   0%|          | 0/3466 [00:00<?, ? examples/s]

Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

In [23]:
def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"]) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in data]
    labels = [torch.tensor(item["labels"]) for item in data]


    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

In [24]:
# Check number of unique NER tags
unique_labels = set([label for example in datasets['train']["ner_tags"] for label in example])
print(f"Unique NER tags: {unique_labels}")
print(f"Model's num_labels: {model.config.num_labels}")

Unique NER tags: {0, 1, 2, 3, 4, 5, 6, 7, 8}
Model's num_labels: 9


In [25]:
# Check if any label is outside the valid range
for example in datasets['train']["ner_tags"]:
    for label in example:
        if label < 0 or label >= model.config.num_labels:
            raise ValueError(f"Invalid label found: {label}. Expected range: [0, {model.config.num_labels - 1}]")

In [26]:
# # Set the context window explicitly to 8192 tokens
# ctx_len = 8192
# tokenizer.model_max_length = ctx_len
# model.config.rope_freq_base = (ctx_len / 131_072) * 500_000
# print(model.config.rope_freq_base)

In [27]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj'],  # Only target attention layers
    bias="none",
    task_type="AutoModelForTokenClassification"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 2,473,351,177 || trainable%: 0.0689


In [28]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
)

In [29]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    args=training_args
)

In [30]:
trainer.train()

  0%|          | 0/937 [00:00<?, ?it/s]

{'loss': 0.4446, 'grad_norm': 1.4213411808013916, 'learning_rate': 8.945823911011648e-05, 'epoch': 0.53}


  0%|          | 0/217 [00:00<?, ?it/s]

{'eval_runtime': 40.2222, 'eval_samples_per_second': 86.171, 'eval_steps_per_second': 5.395, 'epoch': 1.0}
{'train_runtime': 1662.5352, 'train_samples_per_second': 9.015, 'train_steps_per_second': 0.564, 'train_loss': 0.311572120690931, 'epoch': 1.0}


TrainOutput(global_step=937, training_loss=0.311572120690931, metrics={'train_runtime': 1662.5352, 'train_samples_per_second': 9.015, 'train_steps_per_second': 0.564, 'total_flos': 2.4996564925345416e+16, 'train_loss': 0.311572120690931, 'epoch': 1.0})

In [31]:
# Input sentence
sentence = "Steve Jobs, the co-founder of Apple Inc., was born in San Francisco, California."

# Tokenize without adding special tokens
tokenized_input = tokenizer(sentence, return_tensors="pt", add_special_tokens=False).to(model.device)

# Get model outputs
outputs = model(**tokenized_input)

# Get predicted labels (argmax over logits)
predicted_labels = outputs.logits.argmax(-1)[0]

# Inverted label map (assuming label_map is defined elsewhere)
label_map_inverted = {v: k for k, v in label_map.items()}

# Initialize variables to store named entities
named_entities = []
current_entity_tokens = []
current_label = None

# Iterate over tokens and predicted labels
for token_id, label_id in zip(tokenized_input["input_ids"][0], predicted_labels):
    token = tokenizer.decode([token_id])
    label = label_map_inverted[label_id.item()]

    # Skip 'O' labels (non-entity tokens)
    if label == "O":
        if current_entity_tokens:
            # Append the current entity and its label to the list
            named_entities.append((" ".join(current_entity_tokens).strip(), current_label))
            current_entity_tokens = []
            current_label = None
        continue

    # Handle subword tokens (tokens starting with '##')
    if token.startswith("##"):
        current_entity_tokens[-1] += token[2:]  # Append subword to the last token
    else:
        # If it's a new entity or different from the current one, append the previous entity first
        if not current_entity_tokens or label.split("-")[0] == "B" or label != current_label:
            if current_entity_tokens:
                named_entities.append((" ".join(current_entity_tokens).strip(), current_label))
            current_entity_tokens = [token]  # Start a new entity
        else:
            current_entity_tokens.append(token)  # Continue appending to the current entity

        current_label = label

# Append any remaining entity at the end
if current_entity_tokens:
    named_entities.append((" ".join(current_entity_tokens).strip(), current_label))

# Print results
print("Example 1:", sentence)
print("####")
print("Named Entities:")
for entity, label in named_entities:
    print(f"{entity}: {label}")

Example 1: Steve Jobs, the co-founder of Apple Inc., was born in San Francisco, California.
####
Named Entities:
Steve: B-PER
Apple: B-ORG
San: B-LOC
Francisco: B-LOC
California: B-LOC
